In [209]:
import nltk

from sklearn.datasets import fetch_20newsgroups
from nltk.tokenize import word_tokenize #Used to extract words from documents
# from nltk.stem import WordNetLemmatizer #Used to lemmatize words

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics

from nltk.stem import WordNetLemmatizer, PorterStemmer

# Загрузка необходимых ресурсов
# nltk.download('wordnet')
# nltk.download('omw-1.4')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('punkt')


from sklearn.cluster import KMeans

import sys
from time import time

import pandas as pd
import numpy as np

import spacy
nlp = spacy.load('en_core_web_trf') 

In [210]:
categories = [
    "alt.atheism",
    "comp.graphics",
    "comp.sys.ibm.pc.hardware",
    "comp.sys.mac.hardware",
    "comp.windows.x",
    "misc.forsale",
    "rec.autos",
    "rec.motorcycles",
    "rec.sport.baseball",
    "rec.sport.hockey",
    "sci.crypt",
    "sci.electronics",
    "sci.med",
    "sci.space",
    "soc.religion.christian",
    "talk.politics.guns",
    "talk.politics.mideast",
    "talk.politics.misc",
    "talk.religion.misc",
    "comp.os.ms-windows.misc",
]
data = fetch_20newsgroups(categories=categories, remove=('headers', 'footers', 'quotes'))
y_true = data.target
# y_true = newsgroups.targetdata = fetch_20newsgroups(subset='train', categories=['alt.atheism', 'sci.med', 'sci.space'], 
#                              shuffle=False, remove=('headers', 'footers', 'quotes'))

# df = pd.DataFrame(data=data.data)

# len(df)
# print(type(df))
# # df.data[0]
# df.head(3)

In [211]:
df = pd.DataFrame()
df['text'] = data.data
len(data.data)


11314

In [212]:
df['target'] = [categories[x] for x in data.target]
df.head()
print(df['target'][1], df['text'][1], sep="\n")

comp.windows.x
A fair number of brave souls who upgraded their SI clock oscillator have
shared their experiences for this poll. Please send a brief message detailing
your experiences with the procedure. Top speed attained, CPU rated speed,
add on cards and adapters, heat sinks, hour of usage per day, floppy disk
functionality with 800 and 1.4 m floppies are especially requested.

I will be summarizing in the next two days, so please add to the network
knowledge base if you have done the clock upgrade and haven't answered this
poll. Thanks.


In [213]:
df['text'][1]

"A fair number of brave souls who upgraded their SI clock oscillator have\nshared their experiences for this poll. Please send a brief message detailing\nyour experiences with the procedure. Top speed attained, CPU rated speed,\nadd on cards and adapters, heat sinks, hour of usage per day, floppy disk\nfunctionality with 800 and 1.4 m floppies are especially requested.\n\nI will be summarizing in the next two days, so please add to the network\nknowledge base if you have done the clock upgrade and haven't answered this\npoll. Thanks."

In [214]:
labels = df.target
true_k = len(np.unique(labels)) ## This should be 3 in this example
print(true_k)

20


In [215]:
df_a = df[df['target'] == 'rec.motorcycles']
df_b = df[df['target'] == 'comp.windows.x']
# df_c = df[df['target'] == 'sci.med']
df = pd.concat([df_a, df_b])
df.reset_index(drop=True, inplace=True)
df

,text,target
0,I was wondering if anyone out there could enli...,rec.motorcycles
1,I recently posted an article asking what kind ...,rec.motorcycles
2,\nIt depends on your priorities. A lot of peo...,rec.motorcycles
3,an excellent automatic can be found in the sub...,rec.motorcycles
4,: Ford and his automobile. I need information...,rec.motorcycles
...,...,...
1167,\n\n\nDynamic RAM is not based on flip-flops; ...,comp.windows.x
1168,\nAnd they're more like 1024x1024x8 charging &...,comp.windows.x
1169,\n\n\n\n Apple has really confused people w...,comp.windows.x
1170,Here is the story:\nI have a network with 4 Ma...,comp.windows.x


In [194]:
print(df['target'][0], df['text'][0], sep="\n")

alt.atheism

Don't be so sure.  Look what happened to Japanese citizens in the US during
World War II.  If you're prepared to say "Let's round these people up and
stick them in a concentration camp without trial", it's only a short step to
gassing them without trial.  After all, it seems that the Nazis originally
only intended to imprison the Jews; the Final Solution was dreamt up partly
because they couldn't afford to run the camps because of the devastation
caused by Goering's Total War.  Those who weren't gassed generally died of
malnutrition or disease.



In [216]:
# Токенизация
def number_normalizer(tokens):
    """Map all numeric tokens to a placeholder.

    For many applications, tokens that begin with a number are not directly
    useful, but the fact that such a token exists can be relevant.  By applying
    this form of dimensionality reduction, some methods may perform better.
    """
    return ("#NUMBER" if token[0].isdigit() else token for token in tokens)

for i in range(len(df['text'])):
    tokenized_text = word_tokenize(df['text'][i])
    # tokenized_text = number_normalizer(tokenized_text)
    tokenized_text = [' '.join([w for w in tokenized_text if w.isalpha() and len(w)>1])]
    df['text'][i] = ' '.join([f.lower() for f in tokenized_text])
    
df

,text,target
0,was wondering if anyone out there could enligh...,rec.motorcycles
1,recently posted an article asking what kind of...,rec.motorcycles
2,it depends on your priorities lot of people pu...,rec.motorcycles
3,an excellent automatic can be found in the sub...,rec.motorcycles
4,ford and his automobile need information on wh...,rec.motorcycles
...,...,...
1167,dynamic ram is not based on there basically si...,comp.windows.x
1168,and they more like charging discharging capaci...,comp.windows.x
1169,apple has really confused people with this who...,comp.windows.x
1170,here is the story have network with macs on lo...,comp.windows.x


In [196]:
print(df['target'][0], df['text'][0], sep="\n")

alt.atheism
do be so sure look what happened to japanese citizens in the us during world war ii if you prepared to say let round these people up and stick them in concentration camp without trial it only short step to gassing them without trial after all it seems that the nazis originally only intended to imprison the jews the final solution was dreamt up partly because they could afford to run the camps because of the devastation caused by goering total war those who were gassed generally died of malnutrition or disease


In [217]:
def remove_names(text):
    doc = nlp(text)
    newString = text
    for e in reversed(doc.ents):
        if e.label_ == "PERSON": # Only if the entity is a PERSON
            newString = newString[:e.start_char] + newString[e.start_char + len(e.text):]
    return newString


for i in range(len(df['text'])):
    df['text'][i] = remove_names(df['text'][i])
df

,text,target
0,was wondering if anyone out there could enligh...,rec.motorcycles
1,recently posted an article asking what kind of...,rec.motorcycles
2,it depends on your priorities lot of people pu...,rec.motorcycles
3,an excellent automatic can be found in the sub...,rec.motorcycles
4,and his automobile need information on whethe...,rec.motorcycles
...,...,...
1167,dynamic ram is not based on there basically si...,comp.windows.x
1168,and they more like charging discharging capaci...,comp.windows.x
1169,apple has really confused people with this who...,comp.windows.x
1170,here is the story have network with macs on lo...,comp.windows.x


In [218]:
import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import wordnet

# Загрузка необходимых ресурсов
# nltk.download('wordnet')
# nltk.download('omw-1.4')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('punkt')

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

# Инициализация лемматизатора и стеммера
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

for i in range(len(df['text'])):
    # stemmed_tokens = [stemmer.stem(token) for token in tokens]
    lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in df['text'][i].split()]
    df['text'][i] = " ".join(lemmatized_tokens)


In [199]:
print(df['target'][0], df['text'][0], sep="\n")

alt.atheism
do be so sure look what happen to japanese citizen in the u during world war ii if you prepared to say let round these people up and stick them in concentration camp without trial it only short step to gas them without trial after all it seem that the nazi originally only intend to imprison the jew the final solution be dreamt up partly because they could afford to run the camp because of the devastation cause by goering total war those who be gas generally die of malnutrition or disease


In [219]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

for i in range(len(df['text'])):
    df['text'][i] = [word for word in df['text'][i].split() if word not in stop_words]
    df['text'][i] = " ".join(str(x) for x in df['text'][i])

df.to_csv('preprocessed_docs.csv', index=False)

df

,text,target
0,wonder anyone could enlighten car saw day spor...,rec.motorcycles
1,recently post article ask kind rate single mal...,rec.motorcycles
2,depends priority lot people put high priority ...,rec.motorcycles
3,excellent automatic found subaru legacy switch...,rec.motorcycles
4,automobile need information whether ford parti...,rec.motorcycles
...,...,...
1167,dynamic ram base basically single transistor c...,comp.windows.x
1168,like charge discharge capacitor dram simm,comp.windows.x
1169,apple really confuse people whole thing think ...,comp.windows.x
1170,story network mac localtalk one ethernet card ...,comp.windows.x


In [201]:
df['target'][0] 
df['text'][0]

'sure look happen japanese citizen u world war ii prepared say let round people stick concentration camp without trial short step gas without trial seem nazi originally intend imprison jew final solution dreamt partly could afford run camp devastation cause goering total war gas generally die malnutrition disease'

In [220]:
for i in range(len(df['text'])):
    df['text'][i] = df['text'][i].split()
df

,text,target
0,"[wonder, anyone, could, enlighten, car, saw, d...",rec.motorcycles
1,"[recently, post, article, ask, kind, rate, sin...",rec.motorcycles
2,"[depends, priority, lot, people, put, high, pr...",rec.motorcycles
3,"[excellent, automatic, found, subaru, legacy, ...",rec.motorcycles
4,"[automobile, need, information, whether, ford,...",rec.motorcycles
...,...,...
1167,"[dynamic, ram, base, basically, single, transi...",comp.windows.x
1168,"[like, charge, discharge, capacitor, dram, simm]",comp.windows.x
1169,"[apple, really, confuse, people, whole, thing,...",comp.windows.x
1170,"[story, network, mac, localtalk, one, ethernet...",comp.windows.x


In [221]:
df = df[df['text'].apply(lambda l: len(l) > 0)]
df.shape


(1118, 2)

In [222]:
%%time
# обучаем Word2Vec на наших данных
import multiprocessing
from gensim.models import Word2Vec


cores = multiprocessing.cpu_count()
print(f'Ядер: {cores}')

model = Word2Vec(df['text'],
                  min_count=5,
                  window=5,
                  vector_size=100,
                  sample=1e-3,
                  alpha=0.025,
                  min_alpha=0.0001,
                  negative=5,
                  workers=cores-1,
                  epochs=5)

print(f'Размер словаря: {len(model.wv.vectors)}')
print(f'Словарь: {model.wv.key_to_index}')

Ядер: 8
Размер словаря: 2100
Словарь: {'car': 0, 'get': 1, 'use': 2, 'would': 3, 'one': 4, 'drive': 5, 'like': 6, 'mac': 7, 'problem': 8, 'know': 9, 'go': 10, 'well': 11, 'make': 12, 'work': 13, 'apple': 14, 'also': 15, 'new': 16, 'time': 17, 'system': 18, 'could': 19, 'think': 20, 'good': 21, 'anyone': 22, 'say': 23, 'much': 24, 'need': 25, 'look': 26, 'want': 27, 'see': 28, 'speed': 29, 'may': 30, 'card': 31, 'thanks': 32, 'price': 33, 'year': 34, 'come': 35, 'take': 36, 'thing': 37, 'please': 38, 'try': 39, 'two': 40, 'monitor': 41, 'power': 42, 'back': 43, 'people': 44, 'right': 45, 'engine': 46, 'way': 47, 'machine': 48, 'driver': 49, 'since': 50, 'even': 51, 'run': 52, 'question': 53, 'give': 54, 'really': 55, 'sound': 56, 'number': 57, 'dealer': 58, 'memory': 59, 'color': 60, 'call': 61, 'disk': 62, 'post': 63, 'model': 64, 'buy': 65, 'sell': 66, 'someone': 67, 'seem': 68, 'old': 69, 'change': 70, 'first': 71, 'chip': 72, 'scsi': 73, 'send': 74, 'high': 75, 'find': 76, 'sure': 7

In [15]:
# texts_as_wordlists = [x.split(' ') for x in df['text']]
# # print(texts_as_wordlists)
# text_vectors = [model.wv.get_mean_vector(wordlist) for wordlist in texts_as_wordlists]
# print(text_vectors)

In [223]:
df_a = df[df['target'] == 'rec.motorcycles'][:50]
df_b = df[df['target'] == 'comp.windows.x'][:50]
df = pd.concat([df_a, df_b])
df

,text,target
0,"[wonder, anyone, could, enlighten, car, saw, d...",rec.motorcycles
1,"[recently, post, article, ask, kind, rate, sin...",rec.motorcycles
2,"[depends, priority, lot, people, put, high, pr...",rec.motorcycles
3,"[excellent, automatic, found, subaru, legacy, ...",rec.motorcycles
4,"[automobile, need, information, whether, ford,...",rec.motorcycles
...,...,...
641,"[yea, thanks, lot, good, information, newsgrou...",comp.windows.x
642,"[radius, precision, color, video, card, mac, f...",comp.windows.x
643,"[anyone, know, nanao, compatible, popular, mac...",comp.windows.x
644,"[week, ago, post, problem, meg, true, ram, yet...",comp.windows.x


In [37]:
df

,text,target
0,"[sure, look, happened, japanese, citizens, us,...",alt.atheism
1,"[seem, like, little, things, least, orders, wo...",alt.atheism
2,"[deletions, first, infinity, mathematical, con...",alt.atheism
3,"[thought, another, one, bible, definately, lac...",alt.atheism
4,"[yes, also, understanding, majority, islamic, ...",alt.atheism
...,...,...
1110,"[might, problem, video, monitor, instead, many...",comp.sys.ibm.pc.hardware
1111,"[searching, phonetic, truetype, font, windows,...",comp.sys.ibm.pc.hardware
1113,"[official, request, discussion, rfd, creation,...",comp.sys.ibm.pc.hardware
1114,"[nothing, beats, skiing, want, real, fun, holi...",comp.sys.ibm.pc.hardware


In [224]:
X = []
map = {}
df['vectorized'] = ""

for (i, tokens) in df['text'].iteritems():
    vecs = []
    for token in tokens:
        # print(token)
        if model.wv.has_index_for(token):
            vecs.append(model.wv.get_vector(token))
    if vecs:
        vec_avg = np.asarray(vecs).mean(axis=0)
        X.append(vec_avg)
        df['vectorized'][i] = vec_avg
        map[len(X) - 1] = i
    else:
        print(i, tokens)

print(f'Векторов предложений: {len(X)}')


Векторов предложений: 100


In [40]:
# df['text'][7]
# X
# df = []
# df.head(8)
df

,text,target,vectorized
0,"[sure, look, happened, japanese, citizens, us,...",alt.atheism,"[-0.35489327, 0.46809992, -0.076527625, 0.2263..."
1,"[seem, like, little, things, least, orders, wo...",alt.atheism,"[-0.48077837, 0.6430152, -0.12509969, 0.292336..."
2,"[deletions, first, infinity, mathematical, con...",alt.atheism,"[-0.43910798, 0.5831662, -0.11837247, 0.265508..."
3,"[thought, another, one, bible, definately, lac...",alt.atheism,"[-0.40613893, 0.53376985, -0.086722, 0.2599461..."
4,"[yes, also, understanding, majority, islamic, ...",alt.atheism,"[-0.31977788, 0.4211217, -0.061175305, 0.20968..."
...,...,...,...
1110,"[might, problem, video, monitor, instead, many...",comp.sys.ibm.pc.hardware,"[-0.36224288, 0.48566166, -0.088247694, 0.2165..."
1111,"[searching, phonetic, truetype, font, windows,...",comp.sys.ibm.pc.hardware,"[-0.56978196, 0.7661294, -0.19079222, 0.311557..."
1113,"[official, request, discussion, rfd, creation,...",comp.sys.ibm.pc.hardware,"[-0.45096523, 0.6047655, -0.11184164, 0.260900..."
1114,"[nothing, beats, skiing, want, real, fun, holi...",comp.sys.ibm.pc.hardware,"[-0.32326558, 0.42236632, -0.05138647, 0.21445..."


In [225]:
df = df[df['vectorized'].apply(lambda l: len(l) > 0)]
df.shape

(100, 3)

In [42]:
r = [] 
for (i, tokens) in df['vectorized'].iteritems():
    r.append(tokens)

df2 = pd.DataFrame(r)
df2.index = df.index
df2['target']=df['target']


df2.to_csv('files\word2vec3by50.csv', index=True)
df.to_csv('files\preprocessed3by50.csv')
df2.head(8)


,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,target
0,-0.354893,0.468100,-0.076528,0.226385,0.247830,0.040653,-0.146507,0.331224,-0.248520,0.396161,...,0.193366,0.192567,0.025828,0.025863,0.331648,0.000792,-0.210656,-0.039354,0.097044,alt.atheism
1,-0.480778,0.643015,-0.125100,0.292337,0.345799,0.108446,-0.210583,0.333875,-0.296069,0.551135,...,0.258347,0.229829,-0.004990,-0.029753,0.433619,0.018759,-0.227618,-0.051830,0.106234,alt.atheism
2,-0.439108,0.583166,-0.118372,0.265508,0.312333,0.096959,-0.190046,0.304370,-0.265254,0.500772,...,0.236195,0.210110,-0.004490,-0.028976,0.387291,0.011695,-0.207815,-0.052112,0.094935,alt.atheism
3,-0.406139,0.533770,-0.086722,0.259946,0.283312,0.053283,-0.170889,0.374653,-0.278275,0.455121,...,0.219207,0.221063,0.026541,0.025911,0.383689,-0.001525,-0.238711,-0.043844,0.111409,alt.atheism
4,-0.319778,0.421122,-0.061175,0.209681,0.221164,0.026492,-0.128943,0.327613,-0.231931,0.355741,...,0.174303,0.181131,0.030239,0.039840,0.306753,-0.004552,-0.203132,-0.032391,0.092678,alt.atheism
5,-0.375785,0.498474,-0.075357,0.247697,0.258709,0.038594,-0.158257,0.366723,-0.269246,0.418308,...,0.203368,0.207862,0.030153,0.035733,0.357706,-0.005003,-0.231217,-0.040945,0.109710,alt.atheism
6,-0.332732,0.442254,-0.077051,0.213667,0.235627,0.042810,-0.139075,0.305981,-0.228811,0.374757,...,0.182947,0.181086,0.019388,0.019471,0.315539,-0.000781,-0.192693,-0.036208,0.089095,alt.atheism
7,-0.131525,0.173758,-0.019931,0.093283,0.086069,-0.005545,-0.045693,0.168418,-0.103246,0.137400,...,0.071046,0.086841,0.025087,0.038847,0.128616,-0.004796,-0.098400,-0.008775,0.051482,alt.atheism


In [226]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=20, random_state=42)
X_svd = svd.fit_transform(X)

print(X_svd.shape)

r = [] 
for elem in X_svd:
    r.append(elem)

df3 = pd.DataFrame(r)
df3.index = df.index

df3['target'] = df['target']
df3.head(8)

df3.to_csv('to_run\svd_word2vec2by50.csv', index=True)
# df.to_csv('files\preprocessed3by50.csv')



(100, 20)


In [129]:
for i in range(len(df['text'])):
    df['text'][i] = " ".join(df['text'][i])
df

C:\Users\agnes\AppData\Local\Temp\ipykernel_4492\1982565736.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i] = " ".join(df['text'][i])


KeyError: 7

In [62]:
df_a = df[df['target'] == 'alt.atheism'][:50]
df_b = df[df['target'] == 'rec.motorcycles'][:50]
df = pd.concat([df_a, df_b])
# print(df.shape)

(100, 2)


In [167]:
vectorizer = TfidfVectorizer(strip_accents='unicode') ## Corpus is in English
X_TFIDF = vectorizer.fit_transform(df['text'])
X_TFIDF

<146x3074 sparse matrix of type '<class 'numpy.float64'>'
	with 6869 stored elements in Compressed Sparse Row format>

In [168]:
from sklearn.decomposition import TruncatedSVD
print(X_TFIDF.shape)
print(type(X_TFIDF))
# for i in range(len(X_TFIDF[0])):
#     print(*X_TFIDF[i, :].toarray())
svd = TruncatedSVD(n_components=20, random_state=42)
X_svd = svd.fit_transform(X_TFIDF)

print(X_svd.shape)
X_svd

(146, 3074)
<class 'scipy.sparse._csr.csr_matrix'>
(146, 20)


array([[ 0.15557853, -0.03188908, -0.0102702 , ..., -0.03334717,
         0.01815515, -0.01711071],
       [ 0.20150051, -0.05007057,  0.0354893 , ..., -0.01821623,
         0.08703823, -0.09141423],
       [ 0.18282328, -0.03735887, -0.00481265, ...,  0.1046585 ,
         0.1052104 ,  0.00489749],
       ...,
       [ 0.0336363 , -0.00972763, -0.00234169, ..., -0.09475976,
        -0.0016842 ,  0.03050728],
       [ 0.17366446,  0.0315433 , -0.06338141, ...,  0.02110113,
        -0.07777703, -0.01225338],
       [ 0.15252575,  0.02088148, -0.08080737, ..., -0.03885924,
        -0.03348215, -0.02602734]])

In [169]:
df3 = pd.DataFrame(X_svd)
df3.index = df.index
df3['target'] = df['target']
df3

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,target
0,0.155579,-0.031889,-0.010270,0.028186,0.011102,0.152325,-0.058564,0.100517,-0.059582,0.141803,...,0.026455,0.029535,0.140694,0.061780,-0.026793,-0.047697,-0.033347,0.018155,-0.017111,alt.atheism
1,0.201501,-0.050071,0.035489,0.115220,-0.134056,0.004731,-0.078250,0.098542,-0.062694,0.012972,...,0.036232,0.346282,-0.022644,0.170654,0.194956,0.103046,-0.018216,0.087038,-0.091414,alt.atheism
2,0.182823,-0.037359,-0.004813,-0.041430,0.026489,-0.033536,-0.028126,0.082918,-0.044998,0.129889,...,-0.022327,0.219836,-0.117542,-0.082382,0.014209,-0.006509,0.104658,0.105210,0.004897,alt.atheism
3,0.312503,-0.068787,0.178170,0.027004,-0.102148,0.085389,0.016996,0.057415,-0.098103,0.037803,...,0.072298,0.096837,0.185983,-0.024189,0.007031,-0.131568,0.195197,0.020529,-0.117348,alt.atheism
4,0.167044,-0.028265,0.075453,-0.035520,-0.051888,-0.002860,-0.015669,-0.132013,-0.128456,-0.008173,...,0.112155,0.050188,0.045004,-0.009489,0.086117,0.069978,0.170569,0.228520,-0.085840,alt.atheism
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,0.127825,0.012125,-0.092150,-0.104837,0.027029,0.044660,0.085186,0.123391,-0.002293,-0.017054,...,-0.075257,0.004670,-0.015438,0.041522,-0.092128,0.028881,0.109417,0.122742,-0.112205,sci.med
146,0.189705,-0.004489,-0.085091,-0.059996,0.044634,-0.126525,0.065973,-0.129320,-0.022813,-0.018456,...,-0.051792,-0.158901,-0.054697,-0.109088,-0.012646,0.022668,-0.109794,0.057516,0.090568,sci.med
147,0.033636,-0.009728,-0.002342,-0.039905,-0.016252,-0.004866,0.024553,0.042057,-0.069495,-0.014308,...,-0.006118,-0.006590,0.005114,-0.030188,0.063785,0.056764,-0.094760,-0.001684,0.030507,sci.med
148,0.173664,0.031543,-0.063381,-0.185516,-0.085588,-0.055801,0.165985,0.090404,-0.131341,-0.066252,...,-0.000548,-0.091584,0.057945,0.142707,0.073901,0.045263,0.021101,-0.077777,-0.012253,sci.med


In [170]:
res = pd.DataFrame(df3)
res.to_csv('to_run/svd_TFIDF3by50.csv', index=True)
res

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,target
0,0.155579,-0.031889,-0.010270,0.028186,0.011102,0.152325,-0.058564,0.100517,-0.059582,0.141803,...,0.026455,0.029535,0.140694,0.061780,-0.026793,-0.047697,-0.033347,0.018155,-0.017111,alt.atheism
1,0.201501,-0.050071,0.035489,0.115220,-0.134056,0.004731,-0.078250,0.098542,-0.062694,0.012972,...,0.036232,0.346282,-0.022644,0.170654,0.194956,0.103046,-0.018216,0.087038,-0.091414,alt.atheism
2,0.182823,-0.037359,-0.004813,-0.041430,0.026489,-0.033536,-0.028126,0.082918,-0.044998,0.129889,...,-0.022327,0.219836,-0.117542,-0.082382,0.014209,-0.006509,0.104658,0.105210,0.004897,alt.atheism
3,0.312503,-0.068787,0.178170,0.027004,-0.102148,0.085389,0.016996,0.057415,-0.098103,0.037803,...,0.072298,0.096837,0.185983,-0.024189,0.007031,-0.131568,0.195197,0.020529,-0.117348,alt.atheism
4,0.167044,-0.028265,0.075453,-0.035520,-0.051888,-0.002860,-0.015669,-0.132013,-0.128456,-0.008173,...,0.112155,0.050188,0.045004,-0.009489,0.086117,0.069978,0.170569,0.228520,-0.085840,alt.atheism
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,0.127825,0.012125,-0.092150,-0.104837,0.027029,0.044660,0.085186,0.123391,-0.002293,-0.017054,...,-0.075257,0.004670,-0.015438,0.041522,-0.092128,0.028881,0.109417,0.122742,-0.112205,sci.med
146,0.189705,-0.004489,-0.085091,-0.059996,0.044634,-0.126525,0.065973,-0.129320,-0.022813,-0.018456,...,-0.051792,-0.158901,-0.054697,-0.109088,-0.012646,0.022668,-0.109794,0.057516,0.090568,sci.med
147,0.033636,-0.009728,-0.002342,-0.039905,-0.016252,-0.004866,0.024553,0.042057,-0.069495,-0.014308,...,-0.006118,-0.006590,0.005114,-0.030188,0.063785,0.056764,-0.094760,-0.001684,0.030507,sci.med
148,0.173664,0.031543,-0.063381,-0.185516,-0.085588,-0.055801,0.165985,0.090404,-0.131341,-0.066252,...,-0.000548,-0.091584,0.057945,0.142707,0.073901,0.045263,0.021101,-0.077777,-0.012253,sci.med
